In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.options.display.max_info_columns=500
pd.options.display.max_info_rows=550
from tensorflow.keras.losses import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn import set_config; set_config(display='diagram')
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, cross_val_score, GridSearchCV
from xgboost import XGBRegressor

2022-09-09 14:39:01.180351: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-09 14:39:01.180391: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
from carbon_forecast.utils.preproc import load_n_preproc, load_n_preproc_all_avail
df = load_n_preproc_all_avail()

Reading data...
Preproc on power % revenue
Preproc on Sector Revenue
Selecting the year 2013 required...
Dropping rows that are showing NaN on...
-> intensity_scope1CY2013
-> intensity_scope2CY2013
-> intensity_scope3CY2013
-> Revenue2013
Filling employee numbers with mean value... (subject to change later)
Filling EV with mean value... (subject to change later)
Calculating Employees & EV per Revenue...
Filling null Climate Strategy Score with 0...
Cleaning P/E Ratio...
Adding Scope 1 and Scope 2 data together...
Doing some column renaming...
Dataset preproc-ed for the year 2013!👍
Reading data...
Preproc on power % revenue
Preproc on Sector Revenue
Selecting the year 2014 required...
Dropping rows that are showing NaN on...
-> intensity_scope1CY2014
-> intensity_scope2CY2014
-> intensity_scope3CY2014
-> Revenue2014
Filling employee numbers with mean value... (subject to change later)
Filling EV with mean value... (subject to change later)
Calculating Employees & EV per Revenue...
Filli

In [4]:
X = df.drop(columns=["company_name", "TCUID", "year", "intensity_1and2","intensity_scope3"])
y = df["intensity_1and2"]

In [5]:
mean_absolute_error(y,y.mean())

2022-09-09 14:39:08.364836: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-09 14:39:08.364925: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-09 14:39:08.364966: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (VIVOLC): /proc/driver/nvidia/version does not exist
2022-09-09 14:39:08.365451: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(), dtype=float64, numpy=498.3965359160297>

In [6]:
num_col = sorted(X.select_dtypes(include=["int64", "float64"]).columns)
cat_col = sorted(list(set(X.columns) - set(num_col)))


num_transformer = MinMaxScaler()
cat_transformer = OneHotEncoder(handle_unknown='ignore')
preprocessor = ColumnTransformer([
    ('num_tr', num_transformer, num_col),
    ('cat_tr', cat_transformer, cat_col)
],
sparse_threshold=0)
preprocessor

ColumnTransformer(sparse_threshold=0,
                  transformers=[('num_tr', MinMaxScaler(),
                                 ['Abrasive product manufacturing',
                                  'Accounting, tax preparation, bookkeeping, '
                                  'and payroll services',
                                  'Adhesive manufacturing',
                                  'Advertising and related services',
                                  'Air and gas compressor manufacturing',
                                  'Air conditioning, refrigeration, and warm '
                                  'air heating equipment manufacturing',
                                  'Air purification...
                                  'All other textile product mills',
                                  'All other transportation equipment '
                                  'manufacturing',
                                  'Alumina refining and primary aluminum '
                                  'production',
                                  'Aluminum product manufacturing from '
                                  'purchased aluminum',
                                  'Ammunition manufacturing',
                                  'Amusement parks, arcades, and gambling '
                                  'industries',
                                  'Analytical laboratory instrument '
                                  'manufacturing', ...]),
                                ('cat_tr',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['Sector'])])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

In [8]:
# def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
#                        model, param_grid, cv=10, scoring_fit='neg_mean_absolute_percentage_error',
#                        do_probabilities = False):
#     gs = GridSearchCV(
#         estimator=model,
#         param_grid=param_grid, 
#         cv=cv, 
#         n_jobs=-1, 
#         scoring=scoring_fit,
#         verbose=2
#     )
#     fitted_model = gs.fit(X_train_data, y_train_data)
    
#     if do_probabilities:
#       pred = fitted_model.predict_proba(X_test_data)
#     else:
#       pred = fitted_model.predict(X_test_data)
    
#     return fitted_model, pred

In [9]:
# model = XGBRegressor()
# param_grid = {
#     'n_estimators': [400, 700, 1000],
# #     'colsample_bytree': [0.7, 0.8],
#     'max_depth': [3,5,10],
#     'reg_alpha': [1.1, 1.2, 1.3],
#     'reg_lambda': [1.1, 1.2, 1.3],
# #     'subsample': [0.7, 0.8, 0.9]
# }

# model, pred = algorithm_pipeline(X_train_transformed, X_test_transformed, y_train, y_test, model, 
#                                  param_grid, cv=5)

# # Root Mean Squared Error
# print(np.sqrt(-model.best_score_))
# print(model.best_params_)

In [10]:
model = XGBRegressor(n_estimators=1500, max_depth=5, objective="reg:squarederror", eval_metric="mape")
cross_val_score(model, X_train_transformed, y_train, cv=5, scoring="neg_mean_absolute_percentage_error")

array([-27.83040243,  -2.47669294,  -2.62760644,  -1.96605891,
        -2.35351845])

In [11]:
model2 = XGBRegressor(n_estimators=1500, max_depth=10, objective="reg:squarederror", eval_metric="mape")
cross_val_score(model2, X_train_transformed, y_train, cv=5, scoring="neg_mean_absolute_percentage_error")

array([-5.2127186 , -0.76001598, -0.85013595, -1.16857824, -1.73363068])

In [ ]:
model3 = XGBRegressor(n_estimators=1500, max_depth=15, objective="reg:squarederror", eval_metric="mape")
cross_val_score(model3, X_train_transformed, y_train, cv=5, scoring="neg_mean_absolute_percentage_error")